# **Maulo Shop Item Crawling**
- http://o2o.maulo.kr/list/?pageNo=93

## **1 Crawling Function**
using requests, lxml

In [1]:
import requests
from lxml.html import fromstring

class MauloItems:
    
    def __init__(self):
        self.url = "http://o2o.maulo.kr/list/?pageNo="
        
    def items_xpath(self, page_number, xpath='//table[@style="margin:4px;"]'):
        result = []
        response = requests.get(self.url + str(page_number)).content.decode('utf-8')
        items = fromstring(response).xpath(xpath)
        xpaths = [
            './/a[@class="mallttlG"]/text()', # Title
            './/font[@color="333333"]//text()', # Price
            './/font[@color="#888888"]//text()', # User Tags
        ]
        for item in items:
            result.append(["".join(item.xpath(_))  for _ in xpaths])
        return result

## **2 Running the Function**
1 ~ 93 Page function  

In [2]:
result = []
maulo_items = MauloItems()
from tqdm import tqdm
for _ in tqdm(range(1, 94)):
    result += maulo_items.items_xpath(_)

100%|██████████| 93/93 [01:11<00:00,  1.29it/s]


In [9]:
import pandas as pd
result_table = pd.DataFrame(result)
result_table.columns = ['Title', 'Price', 'Tags']
result_table.to_csv("maulo_items.csv", encoding='ms949', index=None)
result_table.head()

,Title,Price,Tags
0,싹독싹독 소독 물티슈,"4,400","코로나,손소독,항균,소독,싹독싹독소독물티슈"
1,에어싹H,"13,000","코로나,손소독,항균,소독"
2,손소독제 소독스프레이 500ml / 20개/Box /일상닥터 살균나라,"8,000","코로나,손소독,항균,소독"
3,휴대용 손소독제 소독 스프레이 60ml / 40개/Box / 일상닥터 살균나라,"4,000","코로나,손소독,항균,소독"
4,일상닥터 버블 손세정제 3종 핸드워시 리필용 200ml / 40개/Box,"2,000","코로나,손소독,항균,소독"


# **Adopting the 'Coupang' item category**
## **1 Read the Text file**

In [2]:
# Read the Data
with open("./data/COUPANG_Category.txt", "r") as f:
    category_text = f.readlines()
category_text = [_.strip()  for _ in category_text]
result_dict = {}
for _ in category_text:
    if _.find(">>") != -1:
        text_key = _
        text_key = text_key.replace(">","")
        text_key = text_key.replace(" ","")
        result_dict[text_key] = []
    else:
        if (len(_)>1) & (_ != "열림"):
            result_dict[text_key].append(_)
result_dict.keys()    

dict_keys(['패션(여성)', '패션(남성)', '스니커즈/슬립온', '뷰티', '출산/유아등', '식품', '주방', '생활용품', '홈인테리어', '가전/디지털', '스포츠/레저', '자동차용품', '도서/음반', '완구/취미', '문구/오피스', '반려용품', '헬스/건강'])

In [3]:
# result_dict
import pandas as pd
item_category = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result_dict.items() ])).fillna('')
# item_category.T.to_csv('categoty.csv', index=None)
item_category.to_csv('categoty.csv', index=None)